In [1]:
import os
import pickle
import requests
import numpy as np
import random

In [2]:
def pad_zero_digit(a: int) -> str:
    a = f'{a:03}'
    return "0"+"0".join(a)


def remove_zero_pad(a: str):
    return a[1::2]


def make_addition_examples_v2(pad=True):
    print('making examples of a+b=c')
    input_file_path = 'add_examples_balanced.txt'
    # if not os.path.exists(input_file_path):
    count = 0
    num_two_digit = 0
    with open(input_file_path, 'w+') as f:
        for a in range(1000):
            for b in range(1000):
                if a < 10 and b < 10: p = 100/100
                elif a < 100 and b < 100: p = 900/(100*100-10*10)
                else: p = 9000/(1000*1000-100*100)
                
                pp = random.uniform(0,1)
                if pp > p:
                    continue
                
                if a < 100 and b < 100:
                    num_two_digit +=1 
                    
                c = a + b
                
                if pad:
                    f.write(f"{a:03}+{b:03}={c:04}\n")
                else:
                    f.write(f"{a}+{b}={c}\n")

            
                count += 1
                if count == 10000:
                    print(f'{num_two_digit} - 2 digit numbers')
                    print(f'{count} number of samples saved to {input_file_path}')
                    return
                    
    print(f'{num_two_digit} - 2 digit numbers')
    print(f'{count} number of samples saved to {input_file_path}')



In [3]:
def numCarryOps(a, b):
    a,b=int(a),int(b)
    def digitSum(n):
        return sum(map(int,str(n)))
    # assert(a >= 0); assert(b >= 0);
    return int((digitSum(a) + digitSum(b) - digitSum(a+b)) / 9)



In [4]:
carry_0_list = []
carry_1_list = []
carry_2_list = []
carry_3_list = []

for a in range(1000):
    for b in range(1000):
            num_carry = numCarryOps(a,b)
            c = a + b

            if num_carry == 0:
                carry_0_list.append(f"{a:03}+{b:03}={c:04}\n")
            elif num_carry == 1:
                carry_1_list.append(f"{a:03}+{b:03}={c:04}\n")
            elif num_carry == 2:
                carry_2_list.append(f"{a:03}+{b:03}={c:04}\n")
            else:
                carry_3_list.append(f"{a:03}+{b:03}={c:04}\n")

print(len(carry_0_list),len(carry_1_list),len(carry_2_list),len(carry_3_list))

166375 358875 338625 136125


In [5]:
random.sample(carry_0_list, 2)

['510+477=0987\n', '430+322=0752\n']

In [7]:
def make_addition_examples_balanced():
    print('making balanced (in terms of # of carry) examples of a+b=c')
    input_file_path = 'add_examples_balanced.txt'
    # if not os.path.exists(input_file_path):
    
    carry_lists = [carry_0_list, carry_1_list, carry_2_list, carry_3_list]
    with open(input_file_path, 'w+') as f:
        count = 0
        for carry_list in carry_lists:
            for example in random.sample(carry_list,2500):
                f.write(example)
                count += 1

    print(f'{count} number of samples saved to {input_file_path}')



In [8]:
# download the tiny shakespeare dataset
input_file_path =  'add_examples_balanced.txt'
# if not os.path.exists(input_file_path):
#     make_addition_examples_v2(pad=True)
make_addition_examples_balanced()



making balanced (in terms of # of carry) examples of a+b=c
10000 number of samples saved to add_examples_balanced.txt


In [9]:
with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data) # 130,023
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# export to bin files
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(f'train_balanced.bin')
val_ids.tofile(f'val_balanced.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(f'meta.pkl', 'wb') as f:
    pickle.dump(meta, f)


length of dataset in characters: 130,000
all the unique characters: 
+0123456789=
vocab size: 13
train has 117,000 tokens
val has 13,000 tokens
